In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_epochs = 4
batch_size = 4
learning_rate = 0.001

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset= torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

data_iter = iter(train_loader)
images, labels = next(data_iter)

class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    output = model(images)
    loss = criterion(output, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss:{loss.item():.2f}')

print('Finished Training')

with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_corrct = [0 for i in range(10)]
  n_class_samples =[0 for i in range(10)]
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    output = model(images)
    _, predicted = torch.max(output, 1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()
    for i in range(batch_size):
      label = labels[i]
      pred = predicted[i]
      if (label == pred):
        n_class_corrct[label] += 1
      n_class_samples[label] += 1
acc = 100.0 * n_correct / n_samples
print(f'Accuracy of the network: {acc}%')
for i in range(10):
  acc = 100.0 * n_class_corrct[i] / n_class_samples[i]
  print(f'Accuracy of {classes[i]}: {acc}%')

Streaming output truncated to the last 5000 lines.
Epoch [4/4], Step [7513/12500], Loss:2.02
Epoch [4/4], Step [7514/12500], Loss:1.39
Epoch [4/4], Step [7515/12500], Loss:1.95
Epoch [4/4], Step [7516/12500], Loss:1.50
Epoch [4/4], Step [7517/12500], Loss:2.40
Epoch [4/4], Step [7518/12500], Loss:1.24
Epoch [4/4], Step [7519/12500], Loss:1.22
Epoch [4/4], Step [7520/12500], Loss:2.14
Epoch [4/4], Step [7521/12500], Loss:1.65
Epoch [4/4], Step [7522/12500], Loss:1.79
Epoch [4/4], Step [7523/12500], Loss:1.91
Epoch [4/4], Step [7524/12500], Loss:1.77
Epoch [4/4], Step [7525/12500], Loss:1.94
Epoch [4/4], Step [7526/12500], Loss:1.47
Epoch [4/4], Step [7527/12500], Loss:1.60
Epoch [4/4], Step [7528/12500], Loss:1.91
Epoch [4/4], Step [7529/12500], Loss:1.68
Epoch [4/4], Step [7530/12500], Loss:1.57
Epoch [4/4], Step [7531/12500], Loss:1.63
Epoch [4/4], Step [7532/12500], Loss:0.78
Epoch [4/4], Step [7533/12500], Loss:2.34
Epoch [4/4], Step [7534/12500], Loss:2.21
Epoch [4/4], Step [7535/1